<a href="https://colab.research.google.com/github/joedmugo10/AI-MoodBoost/blob/main/AI_MoodBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import random
import nltk
import requests
from transformers import AutoModelForCausalLM, AutoTokenizer
from bs4 import BeautifulSoup

# Descarga recursos necesarios de NLTK si no los tienes
nltk.download("punkt")

# Lista de palabras relacionadas con la tristeza
palabras_tristeza = ["triste", "abatido", "desanimado", "melancólico", "desalentado", "deprimido"]

# Función para traducir texto usando la API de Traducción de Google
def translate_text(text, target_language="en"):
    url = "https://translate.googleapis.com/translate_a/single"
    params = {
        "client": "gtx",
        "sl": "auto",
        "tl": target_language,
        "dt": "t",
        "q": text
    }

    response = requests.get(url, params=params)
    translations = response.json()
    translated_text = translations[0][0][0]
    return translated_text

# Función para generar una frase motivacional aleatoria
def generar_frase_motivacional():
    frases_motivacionales = [
        "El éxito es el resultado de la perseverancia.",
        "Cree en ti mismo y en tu capacidad para lograr tus metas.",
        "Cada nuevo día es una oportunidad para crecer y aprender.",
        "Las dificultades son oportunidades disfrazadas.",
        "No importa cuántas veces caigas, lo que importa es cuántas veces te levantes.",
        "El optimismo es la fe que conduce al logro.",
        "Tú eres dueño de tu destino. Toma el control y crea tu propio camino.",
    ]
    return random.choice(frases_motivacionales)

# Instancia del tokenizador y el modelo preentrenado DialoGPT-Medium de Microsoft
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Inicializa el historial del chat
chat_history_ids = None

# Conversación para brindar apoyo emocional y motivacional
print("¡Hola! Soy tu chatbot de apoyo emocional y motivacional. Puedo ayudarte a lidiar con la tristeza.")
print("Si necesitas salir, simplemente escribe 'adiós'.")

while True:
    user_input = input(">> Usuario: ")

    # Verifica si el usuario quiere salir
    if "adiós" in user_input.lower():
        print("Chatbot: ¡Hasta luego! Espero que tengas un día maravilloso.")
        break

    # Traduce la entrada del usuario al inglés
    user_input_english = translate_text(user_input, target_language="en")

    # Verifica si la entrada del usuario contiene palabras relacionadas con la tristeza
    if any(word in user_input.lower() for word in palabras_tristeza):
        # Si hay palabras de tristeza, responde con una frase motivacional
        print("Chatbot: ¡Veo que te sientes triste! Aquí tienes una frase motivacional para ti:")
        print("Chatbot:", generar_frase_motivacional())
    else:
        # Codifica la nueva entrada del usuario en inglés y agrega el token de fin de secuencia (eos_token)
        new_user_input_ids = tokenizer.encode(user_input_english + tokenizer.eos_token, return_tensors='pt')

        # Añade los nuevos tokens de entrada del usuario al historial del chat
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids is not None else new_user_input_ids

        # Genera una respuesta en inglés limitando la longitud total del historial del chat a 1000 tokens
        chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

        # Decodifica la respuesta en inglés
        english_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

        # Traduce la respuesta al español
        spanish_response = translate_text(english_response, target_language="es")

        # Imprime la respuesta en español
        print("Chatbot:", spanish_response)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


¡Hola! Soy tu chatbot de apoyo emocional y motivacional. Puedo ayudarte a lidiar con la tristeza.
Si necesitas salir, simplemente escribe 'adiós'.
